# Fitting in different orbital bases

In this tutorial, we show how one can perform orbit-fits in different coordinate bases amongst the ones supported by `orbitize`. Currently fitting in different bases is only supported in MCMC, so we will use MCMC to perform an orbit-fit in an orbital basis distinct from the default one. For a general introduction to MCMC, be sure to check out the [MCMC Introduction tutorial](https://orbitize.readthedocs.io/en/latest/tutorials/MCMC_tutorial.html) first!

## The "standard" and "XYZ" bases

The default way to define an orbit in `orbitize` is through what we call the 'standard basis', which consists of eight parameters: semi-major axis (sma), eccentricity (ecc), inclination (inc), argument of periastron (aop), position angle of the nodes (pan), epoch of periastron expressed as a fraction of the period past a reference epoch (tau), parallax (plx) and total system mass (mtot). Each orbital element has an associated default prior; to see how to explore and modify these priors check out the [Modifying priors tutorial](http://orbitize.info/en/latest/tutorials/Modifying_Priors.html).

An alternative way to define an orbit is through its position and velocity components in XYZ space for a given epoch; we will call this the 'XYZ basis'. The orbit is thus defined with the array ($x$, $y$, $z$, $\dot{x}$, $\dot{y}$,$\dot{z}$, plx, mtot), with position coordinates measured in AU and velocity components in $\text{km s}^{-1}$. In this basis, the sky-plane coordinates ($x,y$) are the separations of the planet relative to the primary, with the positive $x$ and $y$ directions coinciding with the positive RA and Dec directions, respectively. The $z$ direction is the line-of-sight coordinate, such that movement in the positive $z$ direction causes a redshift. The default priors are uniform all uniform.

## Setting up Sampler in the XYZ basis

The easiest way to run an orbit-fit in an alternative orbital basis in `orbitize` is through the `orbitize.driver.Driver` interface. The process is exactly like initializing a regular `Driver` object, but setting the `fitting_basis` keyword to 'XYZ':

In [ ]:
import numpy as np

import orbitize
from orbitize import driver
import multiprocessing as mp

filename = "{}/ ".format(orbitize.DATADIR) # a file with input in radec since rn it only works for that

# system parameters
num_secondary_bodies = 1
system_mass = 1.75 # [Msol]
plx = 51.44 # [mas]
mass_err = 0.05 # [Msol]
plx_err = 0.12 # [mas]

# MCMC parameters
num_temps = 5
num_walkers = 20
num_threads = mp.cpu_count() # or a different number if you prefer


my_driver = driver.Driver(
    filename, 'MCMC', num_secondary_bodies, system_mass, plx, mass_err=mass_err, plx_err=plx_err, fitting_basis='XYZ',
    mcmc_kwargs={'num_temps': num_temps, 'num_walkers': num_walkers, 'num_threads': num_threads}
)

s = my_driver.sampler

### (Properly) initializing walkers in the XYZ basis

In the standard basis at this point we would be ready to use the `s.run_sampler` method to start the sampling, but with the XYZ basis we have to make sure that all our walkers are initialized in a valid region of parameter space. This is because randomly generated values of ($x$, $y$, $z$, $\dot{x}$, $\dot{y}$, $\dot{z}$) can result in unbound, invalid orbits with, for example, negative eccentricities (which is not cool). This can be easily corrected with the `s.validate_xyz_positions` method:

In [ ]:
s.validate_xyz_positions()

___________________

After this is done, the sampler can be run and the results saved normally:

In [ ]:
total_orbits = 6000 # number of steps x number of walkers (at lowest temperature)
burn_steps = 10 # steps to burn in per walker
thin = 2 # only save every 2nd step

s.run_sampler(total_orbits, burn_steps=burn_steps, thin=thin)
s.results.save_results('my_posterior.hdf5')

## Loading and converting results

You can load the results as you normally would. The orbit posteriors are saved in the `results.post` attribute, and the basis you used for the fit in the `results.fitting_basis` attribute:

In [ ]:
from orbitize import results 

myResults = orbitize.results.Results() # create empty Results object
myResults.load_results('my_posterior.hdf5') 
print('The used basis for the fit was ', myResults.fitting_basis)
print('The posteriors are ', myResults.post)

It is important to note that, as you can probably see, the results will be saved in the XYZ basis, so we might want to convert those back to the standard basis to work with them. For this we can use the `orbitize.conversions` module, which can convert from the standard basis to the XYZ basis and back. To do this, remember that the XYZ basis gives the position and velocity of the planet at a given epoch; so before converting back to standard we need to check the epoch that was used for the fit through the `results.xyz_epochs` attribute:

In [ ]:
print(myResults.xyz_epochs)

Since we only fit for one planet, we only have one epoch. Now, we can use that epoch to convert back to the good old standard basis:

In [ ]:
from orbitize import conversions
xyz_posterior = myResults.post
xyz_epoch = myResults.xyz_epochs

standard_posterior = conversions.xyz_to_standard(xyz_epoch, xyz_posterior)

print('My posterior in standard basis is ', standard_posterior)

And we're done! Enjoy the XYZ basis.